
# _Fine-tuning_ de modelo Qwen
Este notebook realiza o _fine-tuning_ do modelo **Qwen/Qwen2.5-0.5B** utilizando o método **LoRA** (Low-Rank Adaptation), focado no domínio de moda.  
O objetivo é treinar o modelo para responder perguntas específicas sobre moda com base em um conjunto supervisionado de **perguntas e respostas** contidas em um arquivo CSV.

O CSV foi gerado em duas etapas:
1. Transcrição dos vídeos da Curadobia
    - Utilizamos modelos de Speech-to-text para gerar transcrições de diversos vídeos sobre moda produzidos pelos parceiros de projeto.
2. Uso de LLM para estruturar os dados
    - A partir de um `.txt` gerado pela transcrição, utilizamos um LLM para gerar perguntas e respostas baseadas no `.txt`. O processo foi feito em etapas com supervisão humana.

## Estrutura do processo de _fine-tuning_
1. **Preparação do ambiente**  
   - Instalação de dependências necessárias.
   - Montagem do Google Drive para leitura dos dados e salvamento do modelo final.

2. **Processamento de dados**  
   - Leitura do CSV que contém colunas `input` (pergunta) e `output` (resposta).
   - Limpeza e formatação dos dados no formato de chat para treino do modelo.

3. **Treinamento com LoRA**  
   - Aplicação do método LoRA para treinar apenas partes específicas do modelo (projeções de atenção).
   - Treinamento silencioso para evitar poluição visual no notebook.

4. **Avaliação e Salvamento**  
   - Avaliação final do modelo.
   - Mesclagem do backbone com os adapters LoRA em um único modelo e salvamento em `.pkl` e `.safetensors`.

5. **Inferência e Comparação**  
   - Definição de funções para comparar o comportamento do modelo **base** e do modelo **fine-tunado**.
   - Execução de perguntas manuais para observar a melhoria obtida após o fine-tuning.


# Configurações iniciais

Esta célula prepara o ambiente para o treinamento:

- Define seeds para reprodutibilidade dos resultados.
- Configura caminhos absolutos para os arquivos no Google Drive:
  - **CSV** com perguntas e respostas.
  - **Modelo final** salvo em `.pkl`.
- Monta o Google Drive (Colab) para acesso aos dados.
- Seleciona automaticamente GPU, se disponível.
- Carrega o tokenizer do Qwen e ajusta o token de padding.
- Define o texto de sistema (`SYSTEM_PREFIX`), que estabelece o contexto do modelo como consultora de moda.


In [ ]:
!pip install -U -q transformers peft accelerate datasets

: 

In [ ]:
# ===== Imports & Config =====
import os, re, json, random, pickle, sys, platform
from pathlib import Path
from typing import Dict

import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
)
from peft import LoraConfig, get_peft_model
from safetensors.torch import save_file

# Seeds / Modelo base
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
BASE_MODEL = "Qwen/Qwen2.5-0.5B"

# Caminhos absolutos
DATA_CSV    = "/content/drive/Shareddrives/nsync_m11/sprint3/perguntas_respostas_1000.csv"
PICKLE_PATH = "/content/drive/Shareddrives/nsync_m11/sprint3/qwen_fine_tuned.pkl"
SAFETENSORS_PATH  = "/content/drive/Shareddrives/nsync_m11/sprint3/qwen_fine_tuned.safetensors"

# Monta Google Drive (modelo foi desenvolvido usando o Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Tokenizer
tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# Tom do sistema
SYSTEM_PREFIX = "Você é uma consultora de moda brasileira. Responda de forma clara, objetiva e elegante."


# Leitura e limpeza do CSV

- Lê o CSV que contém as colunas:
  - **input**: pergunta.
  - **output**: resposta.
- Valida a presença dessas colunas.
- Remove linhas vazias ou com dados inválidos.
- Renomeia a coluna `input` para `instruction` para padronização interna do pipeline.


In [ ]:
csv_path = Path(DATA_CSV)
assert csv_path.exists(), f"CSV não encontrado: {DATA_CSV}"

df = pd.read_csv(csv_path)
assert "input" in df.columns and "output" in df.columns, f"CSV deve conter colunas 'input' e 'output'. Colunas: {list(df.columns)}"

df = df.rename(columns={"input": "instruction", "output": "output"})
df["instruction"] = df["instruction"].astype(str).str.strip()
df["output"] = df["output"].astype(str).str.strip()
df = df[(df["instruction"] != "") & (df["output"] != "")].reset_index(drop=True)


# Conversão para formato de chat

Transforma cada exemplo do CSV em um formato de **conversa estruturada**, utilizado no treino e na inferência.  
O formato final será:

```<|system|>
{SYSTEM_PREFIX}
<|user|>
{instruction}
<|assistant|>
{output}
```

Esse formato garante consistência no processo, facilitando que o modelo aprenda a diferenciar perguntas de respostas.



In [ ]:
# Opcional: limite para testes rápidos (None = usa tudo)
MAX_EXAMPLES = None
df_trainable = df.iloc[:MAX_EXAMPLES] if MAX_EXAMPLES else df

def to_text_row(instr: str, output: str, input_text: str = "") -> str:
    if input_text:
        return (
            f"<|system|>\n{SYSTEM_PREFIX}\n"
            f"<|user|>\n{instr}\n\nContexto:\n{input_text}\n"
            f"<|assistant|>\n{output}\n"
        )
    else:
        return (
            f"<|system|>\n{SYSTEM_PREFIX}\n"
            f"<|user|>\n{instr}\n"
            f"<|assistant|>\n{output}\n"
        )

texts = [to_text_row(r["instruction"], r["output"]) for _, r in df_trainable.iterrows()]


# Criação do dataset e tokenização

1. Cria um objeto `Dataset` com todos os exemplos.
2. Divide em **treino (90%)** e **validação (10%)** para avaliação durante e após o treinamento.
3. Tokeniza os textos:
   - Limite máximo de 1024 tokens por amostra.
   - Truncamento automático de textos muito longos.


In [ ]:
ds = Dataset.from_dict({"text": texts})
ds = ds.train_test_split(test_size=0.1, seed=SEED)

def tokenize(batch):
    return tok(batch["text"], truncation=True, max_length=1024)

ds_tok = ds.map(tokenize, batched=True, remove_columns=["text"])

# Carregamento do modelo e aplicação do LoRA

- Carrega o modelo base **Qwen/Qwen2.5-0.5B**.
- Aplica **LoRA** (Low-Rank Adaptation), que permite treinar apenas partes específicas do modelo:
  - Camadas de projeção da atenção (`q_proj`, `k_proj`, `v_proj`, `o_proj`).
- Essa abordagem reduz o custo computacional e mantém a maior parte dos parâmetros congelados.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_cfg)

# Configuração do treinamento

- Define os hiperparâmetros principais:
  - `batch_size`, `epochs`, `learning_rate`.
  - Estratégias de logging e salvamento desativadas para manter o notebook limpo.
- Utiliza `DataCollatorForLanguageModeling` configurado para **modelos causais** (`mlm=False`).


In [ ]:
os.environ["WANDB_DISABLED"] = "true"

args = TrainingArguments(
    output_dir="./_tmp_session_only",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_strategy="no",
    bf16=torch.cuda.is_available(),
)

collator = DataCollatorForLanguageModeling(tok, mlm=False)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_tok["train"],
    eval_dataset=ds_tok["test"],
    data_collator=collator
)


# Treinamento do modelo

Inicia o processo de treinamento com LoRA aplicado ao backbone Qwen.


In [ ]:
train_result = trainer.train()

# Merge do modelo e salvamento

- Após o treino, os pesos do LoRA são **mesclados** ao modelo base.  
- O resultado final é salvo em **dois formatos**:
  - `.pkl` – compatível com carregamento via `pickle`, útil para scripts Python simples.  
  - `.safetensors` – formato mais seguro e eficiente, recomendado para uso em produção ou compartilhamento.  




In [ ]:
try:
    merged = model.merge_and_unload()
except AttributeError:
    merged = model.base_model.merge_and_unload()

# Cria diretórios se não existirem
out_dir = Path(PICKLE_PATH).parent
out_dir.mkdir(parents=True, exist_ok=True)

# --- Salvar em .pkl ---
with open(PICKLE_PATH, "wb") as f:
    pickle.dump(merged.state_dict(), f)

# --- Salvar em .safetensors ---
merged.save_pretrained(out_dir, safe_serialization=True)
tok.save_pretrained(out_dir)

print(f"Modelo salvo em:\n- {PICKLE_PATH}\n- {out_dir}/model.safetensors")


# Funções de inferência

Define funções auxiliares para:
- Carregar o modelo base para comparação.
- Montar prompts de entrada no formato esperado.


In [ ]:
# Modelo base para comparação
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
base_model.eval()
merged.eval()

def build_prompt(user_msg: str) -> str:
    return f"<|system|>\n{SYSTEM_PREFIX}\n<|user|>\n{user_msg}\n<|assistant|>\n"

def generate_only_new(model, prompt: str, max_new_tokens=200, temperature=0.7, top_p=0.9, seed=123):
    torch.manual_seed(seed)
    inputs = tok(prompt, return_tensors="pt").to(next(model.parameters()).device)
    input_len = inputs["input_ids"].shape[-1]
    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tok.eos_token_id,
        eos_token_id=tok.eos_token_id
    )
    new_tokens = out[0][input_len:]
    text = tok.decode(new_tokens, skip_special_tokens=True).strip()
    if "<|user|>" in text:
        text = text.split("<|user|>")[0].strip()
    return text


# Comparação entre modelo base e fine-tunado

Permite comparar a performance do modelo base e do modelo ajustado em **3 perguntas escolhidas manualmente**:

1. `Q1`, `Q2` e `Q3` são perguntas presentes no CSV.
2. A célula exibirá lado a lado:
   - Resposta do Qwen base.
   - Resposta do Qwen após fine-tuning.


In [ ]:
Q1 = "Quais marcas oferecem boas regatas básicas?"
Q2 = "Como combinar tênis em looks elegantes?"
Q3 = "Que cor de bolsa combina com roupas escuras além do preto?"

tests = [q for q in [Q1, Q2, Q3] if isinstance(q, str) and q.strip()]

# Executa comparação para cada pergunta preenchida
results = []
for idx, q in enumerate(tests, 1):
    prompt = build_prompt(q)
    base_ans = generate_only_new(base_model, prompt, max_new_tokens=220, temperature=0.1, top_p=0.9, seed=idx*11)
    ft_ans   = generate_only_new(merged,     prompt, max_new_tokens=220, temperature=0.1, top_p=0.9, seed=idx*11)
    results.append((q, base_ans, ft_ans))

# Exibe resultados de forma simples
for i, (q, base_ans, ft_ans) in enumerate(results, 1):
    print("="*80)
    print(f"[Pergunta {i}]")
    print(q)
    print("-"*80)
    print("[QWEN BASE]")
    print(base_ans)
    print("-"*80)
    print("[QWEN FINE-TUNING]")
    print(ft_ans)
print("="*80 if results else "")


# Análise Semântica Avançada de Clusters

Esta seção implementa uma análise semântica avançada para visualizar e quantificar clusters de perguntas similares.
O objetivo é entender como o modelo organiza semanticamente as perguntas e identificar padrões de classificação.

## Funcionalidades:
1. **Extração de Embeddings**: Extrai representações semânticas das perguntas usando o modelo fine-tunado
2. **Múltiplas Técnicas de Redução de Dimensão**: t-SNE, UMAP e PCA
3. **Algoritmos de Clustering**: K-means, DBSCAN e Clustering Hierárquico
4. **Visualizações Interativas**: Gráficos 2D e 3D com Plotly
5. **Métricas de Avaliação**: Silhouette Score, Calinski-Harabasz Index
6. **Análise de Variações Linguísticas**: Identificação de perguntas similares com diferentes formulações

In [ ]:
CLUSTERING_CONFIG = {
    "max_samples": 2000,
    "embedding_strategy": "last_hidden_state",
    "tsne_perplexity": 30,
    "tsne_learning_rate": 200,
    "umap_n_neighbors": 15,
    "umap_min_dist": 0.1,
    "n_clusters_range": range(2, 21),
    "random_state": SEED
}

In [ ]:
# Extrai representações semânticas das perguntas usando o modelo fine-tunado
# Converte texto em vetores numéricos que capturam o significado semântico
def extract_question_embeddings(model, tokenizer, questions, strategy="last_hidden_state", max_length=512):

    model.eval()
    embeddings = []

    with torch.no_grad():
        for question in questions:
            # Tokeniza a pergunta
            inputs = tokenizer(
                question,
                return_tensors="pt",
                truncation=True,
                max_length=max_length,
                padding=True
            ).to(next(model.parameters()).device)

            # Obtém outputs do modelo
            outputs = model(**inputs, output_hidden_states=True)

            if strategy == "last_hidden_state":
                # Usa o último token (CLS-like)
                embedding = outputs.hidden_states[-1][:, -1, :].float().cpu().numpy()
            elif strategy == "mean":
                # Média dos tokens (ignorando padding)
                attention_mask = inputs["attention_mask"]
                hidden_states = outputs.hidden_states[-1].float()
                mask = attention_mask.unsqueeze(-1).expand(hidden_states.size()).float()
                masked_embeddings = hidden_states * mask
                sum_embeddings = torch.sum(masked_embeddings, dim=1)
                sum_mask = torch.sum(mask, dim=1)
                embedding = (sum_embeddings / sum_mask).cpu().numpy()
            elif strategy == "pooler":
                # Usa pooler se disponível, senão usa last_hidden_state
                if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
                    embedding = outputs.pooler_output.float().cpu().numpy()
                else:
                    embedding = outputs.hidden_states[-1][:, -1, :].float().cpu().numpy()
            else:
                raise ValueError(f"Estratégia '{strategy}' não suportada")

            embeddings.append(embedding[0])

    return np.array(embeddings)

In [ ]:
# Reduz a dimensionalidade dos embeddings para visualização em 2D
# Aplica PCA, t-SNE e UMAP para criar representações visuais dos dados
def reduce_dimensions(embeddings, config):

    # Normalização
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)

    reductions = {}

    # PCA
    print("Aplicando PCA...")
    pca = PCA(n_components=50, random_state=config["random_state"])
    reductions["pca_50d"] = pca.fit_transform(embeddings_scaled)

    pca_2d = PCA(n_components=2, random_state=config["random_state"])
    reductions["pca_2d"] = pca_2d.fit_transform(embeddings_scaled)

    # t-SNE
    print("Aplicando t-SNE...")
    tsne = TSNE(
        n_components=2,
        perplexity=config["tsne_perplexity"],
        learning_rate=config["tsne_learning_rate"],
        random_state=config["random_state"],
        max_iter=1000
    )
    reductions["tsne_2d"] = tsne.fit_transform(embeddings_scaled)

    # UMAP
    print("Aplicando UMAP...")
    umap_reducer = umap.UMAP(
        n_components=2,
        n_neighbors=config["umap_n_neighbors"],
        min_dist=config["umap_min_dist"],
        random_state=config["random_state"]
    )
    reductions["umap_2d"] = umap_reducer.fit_transform(embeddings_scaled)

    return reductions

In [ ]:
# Encontra o número ideal de clusters usando métricas de qualidade
# Testa diferentes números de clusters e escolhe o melhor baseado no Silhouette Score
def find_optimal_clusters(embeddings, config):

    # Normalização
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)

    silhouette_scores = []
    calinski_scores = []
    inertias = []

    for n_clusters in config["n_clusters_range"]:
        kmeans = KMeans(n_clusters=n_clusters, random_state=config["random_state"], n_init=10)
        cluster_labels = kmeans.fit_predict(embeddings_scaled)

        # Silhouette Score
        sil_score = silhouette_score(embeddings_scaled, cluster_labels)
        silhouette_scores.append(sil_score)

        # Calinski-Harabasz Index
        cal_score = calinski_harabasz_score(embeddings_scaled, cluster_labels)
        calinski_scores.append(cal_score)

        # Inertia
        inertias.append(kmeans.inertia_)

    # Encontra o número ótimo
    optimal_sil = config["n_clusters_range"][np.argmax(silhouette_scores)]
    optimal_cal = config["n_clusters_range"][np.argmax(calinski_scores)]

    return {
        "n_clusters_range": list(config["n_clusters_range"]),
        "silhouette_scores": silhouette_scores,
        "calinski_scores": calinski_scores,
        "inertias": inertias,
        "optimal_silhouette": optimal_sil,
        "optimal_calinski": optimal_cal
    }

In [ ]:
# Aplica algoritmos de clustering para agrupar perguntas similares
# Usa K-Means, DBSCAN e Clustering Hierárquico para validar os resultados
def apply_clustering_algorithms(embeddings, n_clusters, config):

    # Normalização
    scaler = StandardScaler()
    embeddings_scaled = scaler.fit_transform(embeddings)

    results = {}

    # K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=config["random_state"], n_init=10)
    results["kmeans"] = kmeans.fit_predict(embeddings_scaled)

    # DBSCAN
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    results["dbscan"] = dbscan.fit_predict(embeddings_scaled)

    # Clustering Hierárquico
    hierarchical = AgglomerativeClustering(n_clusters=n_clusters)
    results["hierarchical"] = hierarchical.fit_predict(embeddings_scaled)

    return results

In [ ]:
# Cria as 3 visualizações principais da análise semântica
# Gera gráficos interativos mostrando clusters, densidade e similaridade
def create_main_visualizations(questions, embeddings, reductions, cluster_results, optimal_n_clusters):

    print("Criando visualizações principais...")

    print("Gerando Gráfico 1: Clusters 2D com t-SNE...")
    fig1 = go.Figure()
    colors = px.colors.qualitative.Set3 + px.colors.qualitative.Pastel1 + px.colors.qualitative.Pastel2

    # Adicionar pontos com tamanho baseado na densidade do cluster
    for cluster_id in range(optimal_n_clusters):
        mask = cluster_results["kmeans"] == cluster_id
        if np.any(mask):
            cluster_questions = [questions[i] for i in range(len(questions)) if mask[i]]
            cluster_size = len(cluster_questions)

            # Tamanho dos pontos baseado no tamanho do cluster
            point_size = max(8, min(20, 8 + (cluster_size / 50)))

            fig1.add_trace(
        go.Scatter(
                    x=reductions["tsne_2d"][mask, 0],
                    y=reductions["tsne_2d"][mask, 1],
            mode='markers',
                    marker=dict(
                        size=point_size,
                        color=colors[cluster_id % len(colors)],
                        opacity=0.7,
                        line=dict(width=1.5, color='white'),
                        symbol='circle'
                    ),
                    text=[f"Cluster {cluster_id}: {q[:50]}..." for q in cluster_questions],
                    hovertemplate='<b>%{text}</b><br>X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>',
                    name=f'Cluster {cluster_id} ({cluster_size} perguntas)',
                    legendgroup=f'cluster_{cluster_id}',
                    showlegend=True
                )
            )

    fig1.update_layout(
        title=dict(
            text="Gráfico 1: Visualização dos Clusters Semânticos (t-SNE 2D)",
            font=dict(size=18, family="Arial Black")
        ),
        xaxis_title="t-SNE Dimensão 1",
        yaxis_title="t-SNE Dimensão 2",
        width=1000,
        height=700,
        showlegend=True,
        template="plotly_white",
        margin=dict(l=80, r=80, t=100, b=80),
        legend=dict(
            x=1.02,
            y=1,
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="Black",
            borderwidth=1,
            font=dict(size=12)
        ),
        xaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgray',
            title_font=dict(size=14),
            tickfont=dict(size=12)
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgray',
            title_font=dict(size=14),
            tickfont=dict(size=12)
        )
    )

    fig1.show()

    print("Gerando Gráfico 2: Análise de Densidade...")
    cluster_stats = []
    for cluster_id in range(optimal_n_clusters):
        mask = cluster_results["kmeans"] == cluster_id
        if np.any(mask):
            cluster_embeddings = embeddings[mask]

            from sklearn.metrics.pairwise import euclidean_distances
            distances = euclidean_distances(cluster_embeddings)
            np.fill_diagonal(distances, np.inf)
            avg_distance = np.mean(distances[distances != np.inf])
            from sklearn.metrics.pairwise import cosine_similarity
            similarities = cosine_similarity(cluster_embeddings)
            np.fill_diagonal(similarities, 0)
            avg_similarity = np.mean(similarities[similarities != 0])

            cluster_stats.append({
                'cluster_id': cluster_id,
                'size': np.sum(mask),
                'avg_distance': avg_distance,
                'avg_similarity': avg_similarity,
                'density_score': 1 / (1 + avg_distance)
            })

    cluster_stats = pd.DataFrame(cluster_stats)

    fig2 = go.Figure()
    # Criar cores únicas para cada cluster
    colors = px.colors.qualitative.Set3 + px.colors.qualitative.Pastel1 + px.colors.qualitative.Pastel2

    # Primeiro trace: círculos sem texto, apenas com cores únicas
    fig2.add_trace(
        go.Scatter(
            x=cluster_stats['avg_distance'],
            y=cluster_stats['avg_similarity'],
            mode='markers',
            marker=dict(
                size=cluster_stats['size'] * 2.5,
                color=[colors[int(cid) % len(colors)] for cid in cluster_stats['cluster_id']],
                opacity=0.8,
                line=dict(width=3, color='white'),
                sizemode='diameter',
                sizemin=20
            ),
            hovertemplate='<b>Cluster %{customdata}</b><br>Size: %{marker.size}<br>Avg Distance: %{x:.3f}<br>Avg Similarity: %{y:.3f}<extra></extra>',
            customdata=cluster_stats['cluster_id'],
            name='Clusters',
            showlegend=False
        )
    )

    # Adicionar uma tabela de legenda no canto superior direito
    legend_x = 0.98
    legend_y = 0.95

    # Título da legenda
    fig2.add_annotation(
        x=legend_x - 0.04,
        y=legend_y + 0.03,
        text="<b>Legenda dos Clusters</b>",
        showarrow=False,
        font=dict(size=14, color='black'),
        xanchor="center",
        yanchor="middle"
    )

    for i, (cluster_id, size, avg_dist, avg_sim) in enumerate(zip(
        cluster_stats['cluster_id'],
        cluster_stats['size'],
        cluster_stats['avg_distance'],
        cluster_stats['avg_similarity']
    )):
        y_offset = 0.02 * (len(cluster_stats) - i - 1)

        # Quadrado colorido da legenda
        fig2.add_shape(
            type="rect",
            x0=legend_x - 0.08, y0=legend_y - y_offset - 0.008,
            x1=legend_x - 0.06, y1=legend_y - y_offset + 0.008,
            fillcolor=colors[int(cluster_id) % len(colors)],
            line=dict(width=1, color='white'),
            layer="above"
        )

        # Texto da legenda
        fig2.add_annotation(
            x=legend_x - 0.05,
            y=legend_y - y_offset,
            text=f"Cluster {int(cluster_id)}: {int(size)} perguntas",
            showarrow=False,
            font=dict(size=12, color='black'),
            xanchor="left",
            yanchor="middle"
        )

    fig2.update_layout(
        title=dict(
            text="Gráfico 2: Análise de Densidade e Coesão dos Clusters",
            font=dict(size=18, family="Arial Black")
        ),
        xaxis_title="Distância Média Interna (menor = mais denso)",
        yaxis_title="Similaridade Média Interna (maior = mais coeso)",
        width=1200,
        height=700,
        template="plotly_white",
        margin=dict(l=80, r=200, t=100, b=80),
        xaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgray',
            title_font=dict(size=14),
            tickfont=dict(size=12)
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgray',
            title_font=dict(size=14),
            tickfont=dict(size=12)
        )
    )

    fig2.show()

    print("Gerando Gráfico 3: Heatmap de Similaridade...")
    cluster_centroids = []
    for cluster_id in range(optimal_n_clusters):
        mask = cluster_results["kmeans"] == cluster_id
        if np.any(mask):
            centroid = np.mean(embeddings[mask], axis=0)
            cluster_centroids.append(centroid)
        else:
            cluster_centroids.append(np.zeros(embeddings.shape[1]))

    cluster_centroids = np.array(cluster_centroids)
    from sklearn.metrics.pairwise import cosine_similarity
    similarity_matrix = cosine_similarity(cluster_centroids)

    # Criar matriz de texto com melhor formatação
    text_matrix = []
    for i in range(optimal_n_clusters):
        row = []
        for j in range(optimal_n_clusters):
            if i == j:
                row.append("1.000")  # Diagonal sempre 1.000
            else:
                row.append(f"{similarity_matrix[i][j]:.3f}")
        text_matrix.append(row)

    fig3 = go.Figure(data=go.Heatmap(
        z=similarity_matrix,
        x=[f'C{i}' for i in range(optimal_n_clusters)],  # Labels mais curtos
        y=[f'C{i}' for i in range(optimal_n_clusters)],
        colorscale='RdYlBu_r',
        zmin=0,
        zmax=1,
        text=text_matrix,
        texttemplate="%{text}",
        textfont={"size": 12, "color": "black", "family": "Arial Bold"},
        hovertemplate='<b>Similaridade entre %{y} e %{x}</b><br>Valor: %{z:.3f}<extra></extra>',
        showscale=True,
        colorbar=dict(
            title=dict(
                text="Similaridade",
                side="right",
                font=dict(size=16)
            ),
            tickmode="linear",
            tick0=0,
            dtick=0.2,
            len=0.8,
            thickness=25,
            tickfont=dict(size=14)
        ),
        xgap=2,  # Espaçamento entre células
        ygap=2
    ))

    fig3.update_layout(
        title=dict(
            text="Gráfico 3: Matriz de Similaridade Entre Clusters",
            font=dict(size=18, family="Arial Black")
        ),
        xaxis_title="Clusters",
        yaxis_title="Clusters",
        width=1000,
        height=700,
        template="plotly_white",
        margin=dict(l=80, r=80, t=100, b=80),
        xaxis=dict(
            tickfont=dict(size=14),
            title_font=dict(size=16),
            side="bottom",
            tickangle=0
        ),
        yaxis=dict(
            tickfont=dict(size=14),
            title_font=dict(size=16),
            autorange="reversed",
            tickangle=0
        )
    )

    fig3.show()

    return {
        'cluster_stats': cluster_stats,
        'similarity_matrix': similarity_matrix,
        'cluster_centroids': cluster_centroids
    }


In [ ]:
# Cria visualização de confiança inspirada no código do professor
# Mostra acertos vs erros com tamanho proporcional à confiança da predição
def create_confidence_analysis_plot(questions, embeddings, reductions, cluster_results, confidence_data):

    y_true = cluster_results["kmeans"]  # Clusters verdadeiros
    y_pred = confidence_data['predictions']  # Predições baseadas em similaridade
    y_conf = confidence_data['confidence']  # Confiança

    # Calcular acertos (se predição = cluster verdadeiro)
    is_correct = (y_true == y_pred).astype(int)

    # Criar tamanhos baseados na confiança
    sizes = 20 + 80 * y_conf

    fig4 = go.Figure()

    # Pontos corretos (verde)
    correct_mask = is_correct == 1
    fig4.add_trace(
            go.Scatter(
            x=reductions["tsne_2d"][correct_mask, 0],
            y=reductions["tsne_2d"][correct_mask, 1],
                mode='markers',
            marker=dict(
                size=sizes[correct_mask],
                color='green',
                opacity=0.7,
                line=dict(width=1, color='darkgreen')
            ),
            name='Predições Corretas',
            hovertemplate='<b>Acerto</b><br>Confiança: %{marker.size}<br>X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>'
        )
    )

    # Pontos incorretos (vermelho) - apenas se houver erros
    incorrect_mask = is_correct == 0
    if incorrect_mask.any():
        fig4.add_trace(
            go.Scatter(
                x=reductions["tsne_2d"][incorrect_mask, 0],
                y=reductions["tsne_2d"][incorrect_mask, 1],
                mode='markers',
                marker=dict(
                    size=sizes[incorrect_mask],
                    color='red',
                    opacity=0.7,
                    line=dict(width=1, color='darkred')
                ),
                name='Predições Incorretas',
                hovertemplate='<b>Erro</b><br>Confiança: %{marker.size}<br>X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>'
            )
        )
    else:
        # Se não há erros, adicionar uma anotação informativa
        fig4.add_annotation(
            x=0.5,
            y=0.95,
            xref="paper",
            yref="paper",
            showarrow=False,
            font=dict(size=16, color='green'),
            bgcolor='rgba(255,255,255,0.8)',
            bordercolor='green',
            borderwidth=2
        )

    fig4.update_layout(
        title=dict(
            text="Gráfico 4: Análise de Confiança e Qualidade",
            font=dict(size=18, family="Arial Black")
        ),
        xaxis_title="t-SNE Dimensão 1",
        yaxis_title="t-SNE Dimensão 2",
        width=1000,
        height=700,
        showlegend=True,
        template="plotly_white",
        margin=dict(l=80, r=80, t=100, b=80),
        xaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgray',
            title_font=dict(size=14),
            tickfont=dict(size=12)
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor='lightgray',
            title_font=dict(size=14),
            tickfont=dict(size=12)
        )
    )

    fig4.show()

    # Análise de erros muito confiantes
    high_conf_threshold = np.nanpercentile(y_conf, 85)
    high_conf_errors = (is_correct == 0) & (y_conf >= high_conf_threshold)

    print(f"\n📊 ANÁLISE DE QUALIDADE:")
    print(f"   • Taxa de acerto geral: {(is_correct.mean() * 100):.1f}%")
    print(f"   • Confiança média: {y_conf.mean():.3f}")
    print(f"   • Confiança máxima: {y_conf.max():.3f}")
    print(f"   • Confiança mínima: {y_conf.min():.3f}")

    if incorrect_mask.any():
        print(f"   • Erros muito confiantes (≥{high_conf_threshold:.2f}): {high_conf_errors.sum()}")
        max_error_conf = y_conf[incorrect_mask].max()
        print(f"   • Maior confiança em erro: {max_error_conf:.3f}")
    else:
        print(f"   • Todas as predições estão corretas com confiança média de {y_conf.mean():.3f}")
        max_error_conf = 0.0

    return {
        'accuracy': is_correct.mean(),
        'avg_confidence': y_conf.mean(),
        'high_confidence_errors': high_conf_errors.sum(),
        'max_error_confidence': max_error_conf
    }


In [ ]:
# Analisa confiança e qualidade do modelo baseado na similaridade com centroides
# Simula predições e calcula métricas de confiança para identificar erros
def analyze_model_confidence_and_quality(questions, embeddings, cluster_results, model, tokenizer):


    # Simular predições baseadas na similaridade com centroides
    from sklearn.metrics.pairwise import cosine_similarity

    cluster_labels = cluster_results["kmeans"]
    cluster_centroids = []

    for cluster_id in range(len(np.unique(cluster_labels))):
        mask = cluster_labels == cluster_id
        if np.any(mask):
            centroid = np.mean(embeddings[mask], axis=0)
            cluster_centroids.append(centroid)
        else:
            cluster_centroids.append(np.zeros(embeddings.shape[1]))

    cluster_centroids = np.array(cluster_centroids)

    # Calcular similaridades com todos os centroides
    similarities = cosine_similarity(embeddings, cluster_centroids)

    # Predições baseadas no centroide mais próximo
    y_pred = np.argmax(similarities, axis=1)

    # Confiança baseada na diferença entre o melhor e segundo melhor
    sorted_similarities = np.sort(similarities, axis=1)
    confidence = sorted_similarities[:, -1] - sorted_similarities[:, -2]

    # Normalizar confiança para 0-1
    confidence = (confidence - confidence.min()) / (confidence.max() - confidence.min() + 1e-8)

    return {
        'predictions': y_pred,
        'confidence': confidence,
        'similarities': similarities,
        'cluster_centroids': cluster_centroids
    }


In [ ]:
# Identifica variações linguísticas dentro dos clusters
# Encontra perguntas com significados similares mas formuladas de forma diferente
def analyze_linguistic_variations(questions, embeddings, cluster_labels, threshold=0.8):


    from sklearn.metrics.pairwise import cosine_similarity

    # Calcular similaridade coseno
    similarity_matrix = cosine_similarity(embeddings)

    variations = {}

    for cluster_id in np.unique(cluster_labels):
        if cluster_id == -1:  # Pular outliers do DBSCAN
            continue

        cluster_mask = cluster_labels == cluster_id
        cluster_questions = [questions[i] for i in range(len(questions)) if cluster_mask[i]]
        cluster_indices = np.where(cluster_mask)[0]

        if len(cluster_indices) < 2:
            continue

        # Encontrar pares similares dentro do cluster
        cluster_similarities = similarity_matrix[np.ix_(cluster_indices, cluster_indices)]

        similar_pairs = []
        for i in range(len(cluster_indices)):
            for j in range(i+1, len(cluster_indices)):
                # Evitar comparação da mesma pergunta consigo mesma
                if i != j and cluster_similarities[i, j] >= threshold:
                    similar_pairs.append({
                        "question1": cluster_questions[i],
                        "question2": cluster_questions[j],
                        "similarity": cluster_similarities[i, j]
                    })

        if similar_pairs:
            variations[f"cluster_{cluster_id}"] = {
                "total_questions": len(cluster_questions),
                "similar_pairs": similar_pairs,
                "variation_rate": len(similar_pairs) / (len(cluster_questions) * (len(cluster_questions) - 1) / 2)
            }

    return variations

In [ ]:
# Encontra exemplos representativos de cada cluster
# Seleciona as perguntas mais próximas ao centroide de cada grupo
def analyze_cluster_examples(questions, cluster_labels, embeddings, n_examples=5):


    from sklearn.metrics.pairwise import cosine_similarity

    cluster_examples = {}

    for cluster_id in np.unique(cluster_labels):
        if cluster_id == -1:  # Pular outliers do DBSCAN
            continue

        cluster_mask = cluster_labels == cluster_id
        cluster_questions = [questions[i] for i in range(len(questions)) if cluster_mask[i]]
        cluster_indices = np.where(cluster_mask)[0]
        cluster_embeddings = embeddings[cluster_indices]

        if len(cluster_questions) < 2:
            continue

        # Encontrar o centroide do cluster
        centroid = np.mean(cluster_embeddings, axis=0)

        # Calcular similaridade com o centroide
        similarities = cosine_similarity([centroid], cluster_embeddings)[0]

        # Pegar os exemplos mais próximos ao centroide
        top_indices = np.argsort(similarities)[-n_examples:][::-1]

        cluster_examples[f"cluster_{cluster_id}"] = {
            "total_questions": len(cluster_questions),
            "representative_examples": [
                {
                    "question": cluster_questions[idx],
                    "similarity_to_centroid": similarities[idx]
                }
                for idx in top_indices
            ]
        }

    return cluster_examples

In [ ]:
# Preparar dados
questions = df_trainable["instruction"].tolist()
if CLUSTERING_CONFIG["max_samples"] and len(questions) > CLUSTERING_CONFIG["max_samples"]:
    # Amostragem aleatória para datasets grandes
    np.random.seed(CLUSTERING_CONFIG["random_state"])
    indices = np.random.choice(len(questions), CLUSTERING_CONFIG["max_samples"], replace=False)
    questions = [questions[i] for i in indices]
    print(f"Usando amostra de {len(questions)} perguntas de {len(df_trainable)} total")

print(f"Extraindo embeddings de {len(questions)} perguntas...")

# Extrair embeddings usando o modelo fine-tunado
embeddings = extract_question_embeddings(
    merged, tok, questions,
    strategy=CLUSTERING_CONFIG["embedding_strategy"]
)

print(f"Embeddings extraídos: {embeddings.shape}")

# Reduzir dimensões
reductions = reduce_dimensions(embeddings, CLUSTERING_CONFIG)

# Otimizar número de clusters
optimization_results = find_optimal_clusters(embeddings, CLUSTERING_CONFIG)

print(f"Número ótimo de clusters (Silhouette): {optimization_results['optimal_silhouette']}")
print(f"Número ótimo de clusters (Calinski-Harabasz): {optimization_results['optimal_calinski']}")

# Usar o número ótimo baseado no Silhouette Score
optimal_n_clusters = optimization_results['optimal_silhouette']

# Aplicar algoritmos de clustering
cluster_results = apply_clustering_algorithms(embeddings, optimal_n_clusters, CLUSTERING_CONFIG)

# Criar visualizações principais
visualization_data = create_main_visualizations(questions, embeddings, reductions, cluster_results, optimal_n_clusters)

# Análise adicional
confidence_data = analyze_model_confidence_and_quality(questions, embeddings, cluster_results, merged, tok)
quality_metrics = create_confidence_analysis_plot(questions, embeddings, reductions, cluster_results, confidence_data)

# Analisar variações linguísticas
linguistic_variations = analyze_linguistic_variations(
    questions, embeddings, cluster_results["kmeans"], threshold=0.7
)

# Analisar exemplos representativos dos clusters
cluster_examples = analyze_cluster_examples(
    questions, cluster_results["kmeans"], embeddings, n_examples=3
)


print(f"\n📊 ESTATÍSTICAS GERAIS:")
print(f"   • Total de perguntas analisadas: {len(questions)}")
print(f"   • Dimensão dos embeddings: {embeddings.shape[1]}")
print(f"   • Número ótimo de clusters: {optimal_n_clusters}")

print(f"   • Melhor Silhouette Score: {max(optimization_results['silhouette_scores']):.3f}")
print(f"   • Melhor Calinski-Harabasz Score: {max(optimization_results['calinski_scores']):.3f}")

print(f"\n🔍 DISTRIBUIÇÃO DOS CLUSTERS (K-Means):")
cluster_counts = np.bincount(cluster_results["kmeans"])
for i, count in enumerate(cluster_counts):
    percentage = (count / len(questions)) * 100
    print(f"   • Cluster {i}: {count} perguntas ({percentage:.1f}%)")

print(f"\n📈 QUALIDADE DOS CLUSTERS:")
cluster_stats = visualization_data['cluster_stats']
avg_similarity = cluster_stats['avg_similarity'].mean()
avg_density = cluster_stats['density_score'].mean()
print(f"   • Similaridade média interna: {avg_similarity:.3f}")
print(f"   • Densidade média: {avg_density:.3f}")

print(f"\n🎯 MÉTRICAS DE QUALIDADE:")
print(f"   • Taxa de acerto: {quality_metrics['accuracy']:.1%}")
print(f"   • Confiança média: {quality_metrics['avg_confidence']:.3f}")
print(f"   • Erros muito confiantes: {quality_metrics['high_confidence_errors']}")
print(f"   • Maior confiança em erro: {quality_metrics['max_error_confidence']:.3f}")

print(f"\n🔍 VARIAÇÕES LINGUÍSTICAS IDENTIFICADAS:")
total_variations = 0
for cluster_name, variation_data in linguistic_variations.items():
    cluster_id = cluster_name.split("_")[1]
    similar_pairs = len(variation_data["similar_pairs"])
    total_variations += similar_pairs
    if similar_pairs > 0:
        print(f"   • Cluster {cluster_id}: {similar_pairs} pares similares")

print(f"\n✅ RESUMO FINAL:")
print(f"   • Sistema identifica {total_variations} variações linguísticas")
print(f"   • Taxa de variação média: {(total_variations / len(questions)) * 100:.1f}%")
print(f"   • Clusters bem definidos com Silhouette Score de {max(optimization_results['silhouette_scores']):.3f}")
print(f"   • Taxa de acerto: {quality_metrics['accuracy']:.1%} com confiança média de {quality_metrics['avg_confidence']:.3f}")